In [1]:
# Import modules

import xarray as xr
import sys, os
import numpy as np
import glob
import matplotlib
import pandas as pd

# Import Gen's plotting script
sys.path.append('/home/565/cdr565/code_general/') # Add folder with plotting script
from plotting_maps import acs_plotting_maps
from plotting_maps.acs_plotting_maps import plot_acs_hazard, regions_dict, cmap_dict, tick_dict

# Import script for calculating region statistics
from plotting_maps.acs_area_statistics import acs_regional_stats, regions


In [2]:
# User inputs

file_in_location = '/g/data/ia39/ncra/extratropical_storms/5km/GWLs/' # Location of netcdf hazard data (1D grids)
file_out_location = '/g/data/ia39/ncra/extratropical_storms/5km/GWLs/figures/' # '/home/565/cdr565/code_general/NCRA_figures/' # Location to save figures

hazard_name = 'RX1H' # 'lows', 'RX1D', 'RX5D', 'RX1H'


In [3]:
# Determine variable name and filenames from user inputs in previous cell

if hazard_name == 'lows':
    varname = 'low_freq'
elif hazard_name in ['RX1D','RX5D']:
    varname = 'pr'
elif hazard_name in ['RX1H']:
    varname = 'prhmax'

fnames = glob.glob('{}{}*_MM*ssp370*.nc'.format(file_in_location,hazard_name)) # Create a list of all filenames for specified hazard


In [4]:
# Plot figures - loop through all filenames to create multiple figures

matplotlib.pyplot.close() # Close any figures open from previous session

print(str(len(fnames)) + ' files in list')

for i in np.arange(len(fnames)):
    file_in = fnames[i]
    ds = xr.open_dataset(file_in)
    
    if (varname == 'low_freq') and ('change.nc' not in file_in):
        ds = ds * 100. # Multiply by 100 to change from fraction to percentage
        
    file_out = file_in[len(file_in_location):-2] + 'png'
    csv_file_out = file_in[len(file_in_location):-2] + 'csv'

    
    # Set figure title from input file name
    if 'MM10' in file_in:
        MM_text = '10th percentile'
    elif 'MM90' in file_in:
        MM_text = '90th percentile'
    elif 'MM50' in file_in:
        MM_text = '50th percentile'
        
    if 'ssp370' in file_in:
        ssp_text = 'SSP3-7.0'
        
    if 'GWL12' in file_in:
        GWL_text = 'GWL1.2'
    elif 'GWL15' in file_in:
        GWL_text = 'GWL1.5'
    elif 'GWL20' in file_in:
        GWL_text = 'GWL2.0'
    elif 'GWL30' in file_in:
        GWL_text = 'GWL3.0'
        
    if 'change.nc' in file_in:
        change_text = 'Change at '
    else:
        change_text = ''
        
    set_title = change_text + GWL_text + ': ' + MM_text

    

    # Specify colour map, ticks, cbar label for each variable
    if 'change.nc' in file_in:
        set_cmap = 'anom_coolwarm_r'
        set_ticks = np.array([-50,-40,-30,-25,-20,-15,-10,-5,0,5,10,15,20,25,30,40,50])
        if varname == 'low_freq':
            set_cbar_label = '% change in low frequency\nrelative to GWL1.2'
        else:
            set_cbar_label = '% change in ' + hazard_name + '\nrelative to GWL1.2'
    elif (varname == 'low_freq') and ('change.nc' not in file_in):
        set_cmap = 'pr_1'
        set_ticks = np.array([0.0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.5,3.0,3.5]) # np.arange(0,3.5,0.5)
        set_cbar_label = '% of time influenced by a low'
    elif ((varname == 'pr') or (varname == 'prhmax')) and ('change.nc' not in file_in):
        set_cmap = 'pr_1'
        set_cbar_label = hazard_name + ' (mm)'
        if hazard_name == 'RX1D':
            set_ticks = np.array([0,20,30,40,50,60,70,80,90,100,120,140,160,180]) # np.arange(0,3.5,0.5)
        elif hazard_name == 'RX5D':
            set_ticks = np.array([0,25,50,75,100,150,200,300,400,500,600,800,1000])
        elif hazard_name == 'RX1H':
            set_ticks = np.array([0,8,10,12,14,16,18,20,25,30,35,40,45])


    
    # Use below loop to determine whether to extend colour bar (if values lie outside cbar range)
    set_cbar_extend = "neither"
    if (ds[varname].max() > np.nanmax(set_ticks)) and (ds[varname].min() < np.nanmin(set_ticks)):
        set_cbar_extend = "both"
    elif ds[varname].max() > np.nanmax(set_ticks):
        set_cbar_extend = "max"
    elif ds[varname].min() < np.nanmin(set_ticks):
        set_cbar_extend = "min"
        

    # Create and save figures
    plot_acs_hazard(data = ds[varname],
                    regions = regions_dict['ncra_regions'],
                    cmap = cmap_dict[set_cmap],
                    ticks = set_ticks,
                    cbar_label = set_cbar_label,
                    cbar_extend = set_cbar_extend,
                    title = set_title,
                    dataset_name = "Multi-model CMIP6 bias-adjustment-input",
                    # date_range = '01 January 1960 to 01 January 2015',
                    contour=False,
                    outfile = file_out_location + file_out,
                   );

    print('i = ' + str(i) + ', figure saved as: ' + file_out_location + file_out)

    matplotlib.pyplot.close() # Close figure - remove this line to print figures to screen

    # Calculate and save region stats
    mask_frac = regions.mask_3D_frac_approx(ds)
    dims = ("lat", "lon",)
    da_summary = acs_regional_stats(ds=ds, var=varname, mask=mask_frac, dims=("lat", "lon"), how = ["mean", "min", "max"], outfile=file_out_location + csv_file_out)


print('')
print('Done!')


21 files in list
i = 0, figure saved as: /g/data/ia39/ncra/extratropical_storms/5km/GWLs/figures/RX1H_AGCD-05i_MM10_ssp370_v1-r1_GWL12.png
i = 1, figure saved as: /g/data/ia39/ncra/extratropical_storms/5km/GWLs/figures/RX1H_AGCD-05i_MM50_ssp370_v1-r1_GWL15_change.png
i = 2, figure saved as: /g/data/ia39/ncra/extratropical_storms/5km/GWLs/figures/RX1H_AGCD-05i_MM90_ssp370_v1-r1_GWL15.png
i = 3, figure saved as: /g/data/ia39/ncra/extratropical_storms/5km/GWLs/figures/RX1H_AGCD-05i_MM90_ssp370_v1-r1_GWL15_change.png
i = 4, figure saved as: /g/data/ia39/ncra/extratropical_storms/5km/GWLs/figures/RX1H_AGCD-05i_MM50_ssp370_v1-r1_GWL30_change.png
i = 5, figure saved as: /g/data/ia39/ncra/extratropical_storms/5km/GWLs/figures/RX1H_AGCD-05i_MM90_ssp370_v1-r1_GWL20_change.png
i = 6, figure saved as: /g/data/ia39/ncra/extratropical_storms/5km/GWLs/figures/RX1H_AGCD-05i_MM50_ssp370_v1-r1_GWL20.png
i = 7, figure saved as: /g/data/ia39/ncra/extratropical_storms/5km/GWLs/figures/RX1H_AGCD-05i_MM50_ss